In [215]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 



url = "C:/Users/Nimrod/Documents/GitHub/2nd_try_iron_repo/week4/marketing_customer_analysis_clean.csv"

df = pd.read_csv(url)

In [216]:
df

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,A,2
1,1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,...,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,A,1
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,...,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,2
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,1
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,...,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,A,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10905,10905,FE99816,Nevada,15563.369440,No,Premium,Bachelor,2011-01-19,Unemployed,F,...,7,Personal Auto,Personal L1,Offer3,Web,1214.400000,Luxury Car,Medsize,A,1
10906,10906,KX53892,Oregon,5259.444853,No,Basic,College,2011-01-06,Employed,F,...,6,Personal Auto,Personal L3,Offer2,Branch,273.018929,Four-Door Car,Medsize,A,1
10907,10907,TL39050,Arizona,23893.304100,No,Extended,Bachelor,2011-02-06,Employed,F,...,2,Corporate Auto,Corporate L3,Offer1,Web,381.306996,Luxury SUV,Medsize,A,2
10908,10908,WA60547,California,11971.977650,No,Premium,College,2011-02-13,Employed,F,...,6,Personal Auto,Personal L1,Offer1,Branch,618.288849,SUV,Medsize,A,2


In [217]:
df_num = df.select_dtypes(include='number')

In [218]:
df_num.columns

Index(['unnamed:_0', 'customer_lifetime_value', 'income',
       'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'total_claim_amount', 'month'],
      dtype='object')

In [219]:
df_num = df_num.drop_duplicates()

df_num.isna().any()

unnamed:_0                       False
customer_lifetime_value          False
income                           False
monthly_premium_auto             False
months_since_last_claim          False
months_since_policy_inception    False
number_of_open_complaints        False
number_of_policies               False
total_claim_amount               False
month                            False
dtype: bool

## Spltting & Scalling

In [220]:
# splitting

X = df_num.drop(['total_claim_amount'], axis=1)
y = df_num[['total_claim_amount']]

In [221]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [222]:
# past split scaling 

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)



## Apply linear regression.


In [223]:
lm = LinearRegression()
model = lm.fit(X_train, y_train)



In [224]:
model.coef_


array([[   6.01927432,  -50.62331362, -329.26455107, 1277.58373142,
           6.21138998,  -17.74720841,   -2.46967714,    2.33584909,
          -1.84174558]])

In [225]:
model.intercept_

array([390.97847183])

## Model Interpretation

In [226]:
y_pred

,0
0,514.793892
1,495.915276
2,231.360995
3,319.174191
4,350.298236
...,...
2266,1421.615081
2267,231.148321
2268,246.774066
2269,276.988266


In [227]:
y_pred = model.predict(X_test)

y_pred = pd.DataFrame(y_pred)

In [228]:
y_test = y_test.reset_index(drop=True)

r_df = pd.concat([y_test, y_pred], axis=1) 

r_df = r_df.rename(columns={"total_claim_amount": "y_test", 0: "y_pred"})

In [229]:
r_df.isna().sum()

y_test    0
y_pred    0
dtype: int64

In [230]:
r_df.head(3)

,y_test,y_pred
0,475.423848,122.658353
1,350.400000,350.524852
2,482.400000,415.505553


In [231]:
r_df['residual'] = r_df['y_test']-r_df['y_pred']
r_df.head()

,y_test,y_pred,residual
0,475.423848,122.658353,352.765495
1,350.400000,350.524852,-0.124852
2,482.400000,415.505553,66.894447
3,673.342650,433.532813,239.809837
4,302.400000,154.757902,147.642098


## Model Interpretation

In [232]:
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse
import numpy as np

In [233]:
RMSE = np.sqrt(mse(y_test, y_pred))
MAE = mae(y_test, y_pred)

print(f'\nModel Interpretation\n{"*"*25}\nMAE:{MAE}\nRMSE: {RMSE}\n')


Model Interpretation
*************************
MAE:146.035723236312
RMSE: 201.2579187895068

